In [23]:
import os
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import shutil
import cv2

# ---------------------------
# 1. DEFINIR RUTAS
# ---------------------------
dir_defectos = "WGAN-Generated-images"
dir_sanos = "sanos"
dir_resultados = "sanos_con_defectos"
graficos_dir = "graficos_comparacion"

# ---------------------------
# 2. PREPARAR CARPETAS DE SALIDA
# ---------------------------
for folder in [dir_resultados, graficos_dir]:
    shutil.rmtree(folder, ignore_errors=True)
    os.makedirs(folder, exist_ok=True)

# ---------------------------
# 3. FUNCIONES AUXILIARES
# ---------------------------
def detectar_mascara_verde(imagen_rgba):
    imagen_rgb = imagen_rgba.convert("RGB")
    imagen_np = np.array(imagen_rgb)
    hsv = cv2.cvtColor(imagen_np, cv2.COLOR_RGB2HSV)
    lower_green = np.array([30, 40, 40])
    upper_green = np.array([90, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    kernel = np.ones((3, 3), np.uint8)
    return cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    
def limpiar_bordes_defecto(defecto_img, umbral=30):
    """
    Elimina bordes y colores oscuros de una imagen de defecto.
    Convierte en transparentes los píxeles con RGB menor al umbral dado.

    Parámetros:
    - defecto_img: PIL.Image (modo RGBA o convertido)
    - umbral: valor entre 0 y 255 para definir lo "oscuro"

    Retorna:
    - Imagen con fondo y bordes oscuros transparentes
    """
    defecto_img = defecto_img.convert("RGBA")
    defecto_arr = np.array(defecto_img)

    r = defecto_arr[:, :, 0]
    g = defecto_arr[:, :, 1]
    b = defecto_arr[:, :, 2]

    # Crear máscara de píxeles oscuros
    mascara_oscura = (r < umbral) & (g < umbral) & (b < umbral)

    # Aplicar transparencia a píxeles oscuros
    defecto_arr[mascara_oscura] = [0, 0, 0, 0]

    return Image.fromarray(defecto_arr)
    
def create_soft_mask(defect_arr):
    mask = np.any(defect_arr[:, :, :3] != [0, 0, 0], axis=-1).astype(np.uint8) * 255
    mask_img = Image.fromarray(mask, mode="L").filter(ImageFilter.GaussianBlur(radius=1.5))
    return np.array(mask_img).astype(np.float32) / 255.0

def aplicar_defecto_con_mascara_verde(sano_img, defecto_img):
    sano_img = sano_img.convert("RGBA").resize((128, 128))
    defecto_img=limpiar_bordes_defecto(defecto_img, umbral=40)
    defecto_img = defecto_img.convert("RGBA").resize((128, 128))
    mask_verde = detectar_mascara_verde(sano_img)
    mask_bin = (mask_verde / 255).astype(np.uint8)
    mask_rgba = np.stack([mask_bin] * 4, axis=-1).astype(np.float32)
    defecto_arr = np.array(defecto_img).astype(np.float32)
    defecto_recortado = defecto_arr * mask_rgba
    alpha = create_soft_mask(defecto_recortado)
    alpha = np.stack([alpha] * 4, axis=-1)
    alpha *= mask_rgba
    sano_arr = np.array(sano_img).astype(np.float32)
    combinado = sano_arr * (1 - alpha) + defecto_recortado * alpha
    return Image.fromarray(np.clip(combinado, 0, 255).astype(np.uint8))

# ---------------------------
# 4. PROCESAR: CADA DEFECTO SOBRE TODOS LOS AGUACATES
# ---------------------------
sano_imgs = sorted([f for f in os.listdir(dir_sanos) if f.lower().endswith(('.png', '.jpg'))])
defecto_imgs = sorted([f for f in os.listdir(dir_defectos) if f.lower().endswith(('.png', '.jpg'))])

contador = 0
for d_idx, defecto_file in enumerate(defecto_imgs):
    defecto_path = os.path.join(dir_defectos, defecto_file)
    defecto_img = Image.open(defecto_path)

    for s_idx, sano_file in enumerate(sano_imgs):
        sano_path = os.path.join(dir_sanos, sano_file)
        sano_img = Image.open(sano_path)

        combinado = aplicar_defecto_con_mascara_verde(sano_img, defecto_img)

        nombre_resultado = f"def{d_idx+1:02}_sano{s_idx+1:02}.png"
        combinado.save(os.path.join(dir_resultados, nombre_resultado))

        # Gráfico comparativo
        fig, axs = plt.subplots(1, 3, figsize=(9, 3))
        axs[0].imshow(sano_img.resize((128, 128)))
        axs[0].set_title("Sano")
        axs[1].imshow(defecto_img.resize((128, 128)))
        axs[1].set_title("Defecto")
        axs[2].imshow(combinado)
        axs[2].set_title("Combinado")
        for ax in axs:
            ax.axis("off")
        plt.tight_layout()
        plt.savefig(os.path.join(graficos_dir, f"grafico_def{d_idx+1:02}_sano{s_idx+1:02}.png"))
        plt.close()

        contador += 1

print(f"✅ {contador} combinaciones generadas.")
print(f"Guardadas en: {dir_resultados} y {graficos_dir}")


✅ 630 combinaciones generadas.
Guardadas en: sanos_con_defectos y graficos_comparacion
